In [1]:
# noexport

!typech evaluation_utils.ipynb


[NbConvertApp] Converting notebook evaluation_utils.ipynb to python


In [1]:
from typing import Dict, List, Tuple, Any

In [2]:
from train_utils import *

KeyboardInterrupt: 

In [ ]:
from getsecret import getsecret
import subprocess
import json

In [ ]:
#print(get_models_currently_training())

In [ ]:
def get_models_currently_training():
  output = []
  local_info = json.load(open('current.json'))
  output.append(local_info['base_path'])
  remote_info = json.loads(subprocess.check_output(getsecret('REMOTE_SSH_COMMAND'), shell=True))
  output.append(remote_info['base_path'])
  return output

In [ ]:
from glob import glob
import os
from os import path

In [ ]:
def find_last_completed_epoch(model_name):
  return len(glob(path.join(model_name, 'info_*.json')))


In [ ]:
#print(get_models_with_training_status())


In [ ]:


def get_models_with_training_status():
  output = []
  models_currently_training = set(get_models_currently_training())
  for x in glob('tm_*'):
    model_status = {
      'name': x,
      'epochs_done': find_last_completed_epoch(x),
      'status': 'incomplete'
    }
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      output.append(model_status)
      continue
    status_info = json.load(open(status_file_path))
    if status_info['status'] == 'done':
      #model_status['epochs_done'] = find_last_completed_epoch(x)
      model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    if status_info['status'] == 'training':
      if x in models_currently_training:
        model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    raise Exception('invalid status for model', x, status_info['status'])
    #output.append(model_status)
  return output

In [ ]:
import shutil

def remove_incomplete_models():
  for status_info in get_models_with_training_status():
    if status_info['epochs_done'] == 0 and status_info['status'] == 'incomplete':
      model_name = status_info['name']
      shutil.rmtree(model_name)
      #print(status_info)

remove_incomplete_models()

In [ ]:
def get_parameter_info_list_for_model(model_name):
  parameters_file_path = path.join(model_name, 'parameters.json')
  return json.load(open(parameters_file_path))

def get_parameter_map_for_model(model_name):
  output = {}
  for parameter_info in get_parameter_info_list_for_model(model_name):
    parameter_name = parameter_info['name']
    parameter_value = parameter_info['value']
    output[parameter_name] = parameter_value
  return output

def list_models_finished_training():
  output = []
  for x in glob('tm_*'):
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      continue
    status_info = json.load(open(status_file_path))
    #print(status_info)
    if status_info['status'] != 'done':
      continue
    output.append(x)
  return output

def list_models_matching_parameters(parameters_map):
  output = []
  for model_name in list_models_finished_training():
    parameters_map_cur = get_parameter_map_for_model(model_name)
    is_match = True
    for k,v in parameters_map.items():
      if parameters_map_cur.get(k, None) != v:
        is_match = False
        break
    if is_match:
      output.append(model_name)
  return output


def get_models_with_dev_correct_fraction():
  #for x in list_models_finished_training():
  #  get_parameter_map_for_model(x))
  output = []
  for model_name in list_models_matching_parameters({'enable_current_difficulty': False}):
    for info_file_name in glob(path.join(model_name, 'info_*.json')):
      model_info = json.load(open(info_file_name))
      #print(model_info)
      dev_correct = model_info['dev_correct']
      dev_total = model_info['dev_total']
      dev_correct_fraction = dev_correct / dev_total
      output.append([dev_correct_fraction, info_file_name])
  output.sort()
  output.reverse()
  return output

get_models_with_dev_correct_fraction()

In [ ]:
# true = True
# false = False
# parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]

# train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)



In [ ]:


#def get_current_difficulty(tensor):
#  return tensor['chosen_difficulty']
# will actually always return nothing unless we enabled it in training data
#   feature = tensor['feature']
#   history_length = feature.size()[0]
#   timestep = history_length - 1
#   difficulty_feature_idx = get_feature_index('difficulty')
#   for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
#     if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
#       return difficulty
#   return 'nothing'

def get_most_recently_chosen_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = history_length - 2
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def get_first_chosen_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = 0
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def get_initial_difficulty(feature, parameter_info_list):
  #feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = 0
  #if len(feature) < 2:
  #  return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('initial_difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def always_predict_nothing(feature, parameter_info_list):
  return 'nothing'

def most_common_key_for_counter(c):
  best_key = None
  for k,v in c.items():
    if best_key == None:
      best_key = k
      continue
    if c[k] > c[best_key]:
      best_key = k
  return best_key

def always_predict_most_common_previous(feature, parameter_info_list):
  history_length = feature.size()[0]
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  difficulty_choices = Counter()
  for timestep in range(0, history_length - 1):
    for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
      if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
        difficulty_choices[difficulty] += 1
  output = most_common_key_for_counter(difficulty_choices)
  if output == None:
    return 'nothing'
  return output
  #return 'nothing'

#print(get_current_difficulty(train_data[299]))
#print(get_most_recently_chosen_difficulty(train_data[299]))
# counts = Counter()
# for x in train_data:
#   current_difficulty = get_most_recently_chosen_difficulty(x['feature'])
#   counts[current_difficulty] += 1
# print(counts)

In [ ]:
#print(train_data[0])

In [ ]:
def evaluate_function_model_on_dataset(parameter_info_list, func, dataset, prefix='dev_') -> Dict[str, Any]:
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    #predicted_tensor = make_prediction(model, feature_tensor.cuda())
    #predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty = func(feature_tensor, parameter_info_list)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }


In [ ]:
def evaluate_baseline_model_on_dataset(parameter_info_list, dataset, prefix='dev_'):
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    #predicted_tensor = make_prediction(model, feature_tensor.cuda())
    #predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty = get_most_recently_chosen_difficulty(feature_tensor, parameter_info_list)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }


In [ ]:
#print(evaluate_baseline_model_on_dataset(dev_data))

In [ ]:
every_n_visits_to_dev_evaluation_results : Dict[int, Dict[str, Any]] = {}
every_n_visits_to_train_evaluation_results : Dict[int, Dict[str, Any]] = {}

In [ ]:
true = True
false = False
parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)


In [ ]:
initial_difficulty_stats = evaluate_function_model_on_dataset(parameter_info_list, get_initial_difficulty, dev_data)
print(initial_difficulty_stats)

In [ ]:
first_chosen_difficulty_stats = evaluate_function_model_on_dataset(parameter_info_list, get_first_chosen_difficulty, dev_data)
print(first_chosen_difficulty_stats)

In [ ]:
always_nothing_stats = evaluate_function_model_on_dataset(parameter_info_list, always_predict_nothing, dev_data)
print(always_nothing_stats)

In [ ]:
most_common_previous_stats = evaluate_function_model_on_dataset(parameter_info_list, always_predict_most_common_previous, dev_data)
print(most_common_previous_stats)

In [ ]:
baseline_name_to_stats : Dict[str, Dict[str, Any]] = {}
name_and_func : List[Tuple[str, Any]] = [
  ('initial_difficulty', get_initial_difficulty),
  ('first_chosen', get_first_chosen_difficulty),
  ('always_nothing', always_predict_nothing),
  ('most_common_previous', always_predict_most_common_previous),
]
for [baseline_name,baseline_func] in name_and_func:
  baseline_name_to_stats[baseline_name] = evaluate_function_model_on_dataset(parameter_info_list, baseline_func, dev_data)

In [21]:
print(len(get_users()))

4780


In [24]:
import jsonmemo
print(jsonmemo.get_cache_dirname())

2019_04_08


In [25]:
print(len(get_all_features_data()))

performing computation get_all_features_data.json
e0ea34c81d4b50cddc7bd752
d45b8b82dec8ec3c591fdbd7
f4d95006c663a799d9185576
8d2c9eb27dee2dc85bca705b
4a5f492c987b873871be4c12
78cc8c70b8be7412e00a1575
04c7fb2ae3098eab5ae390b8
8642884bd204eafc660a04a2
9ca53f1f33ac3e25c2e3f3d2
6e055258e3625ab1db57f0fa
9c00827409f8c296d67f78b4
687606ae6354d8b707f064ee
57ddb2820db2de361757d21b
af8454409cde73bead9d1071
eccfe39b9019a3f566346399
f32cf25130f6fa751892960e
8f77439cf19de38abfd4a417
3bc557eda64de5ff9d2e6fca
16043b6d098957d10448c009
edc3437cabf565d7a8c2f96c
88550c3144f8e1a6e84e83e2
3bc201cd34fc67185861245e
472a909dcf35ed6f1bc6da82
f45c319882842376492cffa7
150f93e5b7dea556c943813e
93b71831840fb44e554b6e36
cb266f371364b060f117b6a1
4e69f3aebac131196a8f2912
834ab49ae4351c22a1d49c2d
43a6fdadccd61a51caf5b588
968e70f748b83408bf3a9fa3
f6e9666fa6fd68a8ff0e2781
48df425ff6b2910b961070e5
ffdc2b2d686dbf9c8c621b35
42564d59c50078ce37d551e1
9595707af25dc9cf08d162f2
414492408538c282663be78d
8880741c0eb16b95ba062848


77d96358edb2978a0899bfcc
7669818720e10485946be94c
7837eb10f86c3a8baf835fcc
c7e7e27d6fb15ced618a19d0
bc2a2a9aa64af2b5cd1b403a
27cfcc4681e11b23b54acf63
a8bf6d62f2be3db749ab5c85
235bb31da5dc39b661d12d22
9ee9b7534c9bcafec6a13773
8fcff0a298c6a5bbd6d40487
297f28ab3e26161b316fcf54
256281b12e168695353321f9
60101b72ef16fe7ca8ad510d
828d5664d65b18aa895a3fe5
ac4c7bb9cf37a34143955ba3
222d7f9138114a93b8a039c5
f111a8bfa3fca00051a0ce7c
0665b11d1ffee106cb3c3fb1
ac5bcd553aa0dc6be14da62a
661f66e6625004badc5f99ee
cf2b84b289d13a0fe56a7f71
49ea2283e79afad7e1ede6ff
8a2d1ec53ad08c204a999032
39778195ca201f72bad0e6de
59ab611d4e99391f215e88f2
6f52e207fcf0e4f2a06729b7
8ccda4acf0cfef149ecedc5c
e19c8558d70c302d03c2b557
f17293d808752f791fef8452
5ee1b051445a4c978e8174a2
09d27c86d835df2af5f0be24
d5e2424eae96ecaead6a2d33
0ae7ef8f2467adc9deac8b78
b35f644dbb52e1b9e5211275
78671462e29c9038f675d095
0e722db741caeab896cc7692
e118e25983b0dd2e48723817
679e6fb57eff0b5153dbe71f
baf0f16632ecf91730f0278c
5d2d752f891fe772b8b2bfdf


d8dc3a0d6a7ebf2ec16d067a
272bdfd42db21e042963fa11
59bab175c66b2e618f2640ec
491e5869a2a3df4408399f94
cd35f84b81596b0930f8d9b5
5e4a219b35c059a00c9131e2
714a57df8163e780463ca316
6f886b8df388d9d079cfa16b
84fee669212988443231d39c
9579d2bf5ad99f02dad5a9bf
79df2c00e38d43ff43d06221
cdd70c2b9066b2a94170c3d3
c6b4af032286d5e88a489cf2
f3c3cc381a98828dc9b61084
0db379d9ba899842a34a008c
54323b1146330f4acf7fc81b
5c69f950cbd4884f485874d0
f70fd960bb5c7d6d33606939
099996f56d114c4f4c4ea486
0d0204e61d7e6b1a8a6791a2
a9918d999b83060a554f5337
e8c996497dbb275b5e15ea82
5b4f5ef2f0be064062842764
e5daab2a45c107b7de88b0f7
89af37619a3ea3122e5bcd72
8555d722020fafda44e2a8dd
65f1392b09ac2f74f8ebea80
62485258d69bc769643addf0
2387ee173747161861938bbc
80be56cc32b08e9e67ce8df5
44dd5db05d3842aaac92eb32
244955f7ea721a410642695a
0c9d6b534718c869fee8a93f
2bf317e4a10f93e63634ed98
23cf768829b8d4b1d31e308e
2064b7528590600a33c7cb80
5e7fbe23c2555cac2da675ce
eade7c04d9cce2319eb6950f
5abbc8374811647848518858
e837a3a10e33decff23c9897


f12f63a378a8ce4a1f9e856b
bff440437a5be8e0c175aaf0
9da1d823cec91d3780cbceb2
6751d267a976e214e678b276
44ba0161172ec028af754247
b104f0690eb75c9b284157a3
fce645da940c68947a2865bb
db4815bf423993303baf4a31
d5bedf70921c6159855d6c56
e48b57bc484668fd268fc877
8c8129e93451e4194b8a0705
e901e68c5a9e04b80a111da7
2872b0d0ad0cf96c92f69376
350240ac1aed84fc034ef0d9
5deeac7f63626bfde2338471
9a78251e828c3d8d2ae63a9f
bd106a69ac43a326070997b4
77b741ea3e0c89d5ea3a7e50
68a7150509fe7bd1b41cdc58
aa243450f621903aef8ef38e
9fb33ce5003bfe9345f8ff18
87b94cae027b319b5ede42d4
4a34efaa691756a18af45817
05fb859582ca77a3cc336407
602f037f733be179de683312
b755196673711336b6bb6409
6484007555c64b753670fda8
b2772df727ad33cd34c53847
ab7b7b02751979d396c60f3c
908af43fb9f18489c3ab8fba
fed078a8a181b7dce3608616
0096425d15d512c160337ea2
46020ed0c4c60faec4e26d34
31ee2cf7392a6a9bbfdce343
e0b72fce18d8319ad6cd8d95
fa84c0d7e5f7496020e5fc88
0b2139cbbed6338a4a70c727
df32b5f8171148ac0e003926
3691e034ce57ed1f0a9ad149
ec951df260c6d955a109109e


cb8cb14f9788b83740c86493
2c517d0c479d6c2ae26219a8
4c77b649e0ce8234ad70fbd5
f6b8fcf548ab0fbe0e4b8c8e
f0cc84fcaa742781ac5820f0
2e2f56108dfcf590742d91cd
b5c7e2ab8aa309e0ba44442d
9bb0ee2aa64e7baec92e82f7
629f24f944bdf98f86b8d773
fde1797b33f767bfb4ac7952
5ff9048eab137827fa446740
f92ad8a7c49dc93349aebc69
982078b73e120a8ec0283118
3e449d9e7143f94ed66eebe1
726c92ca624b2fd99a8acdfa
7d1c844cdcbfa6485120d835
0d2483d57c8f634692dbe9c3
d325f40561488b5f450e88bf
ddb10214701439713d69df2b
e9c50b332f319416676f35f7
573ac7c691bc33e5fcdb5670
ff66543aecde06d495f8cf74
23ebdbe291c15ce457280ea0
68a49f75f9422acb39c6c6ab
9b4564be30d09eb0ccfde0fd
84cec107f45cb7851b56ebd8
78e5ed8269ebdd6547fbe032
0e5e859214859b5e6383a1cf
97a4d76bdc56abd8ebfbd2b5
2d1ddf6cf9fc49a17cafc303
d347801411a564e10bba3b59
005648ef03b968927261272c
503da81376a52d6715015404
d91824320387a1de3033de9c
b3dd048a4edefa21d3e47d34
ac5f5234cef7aa0c3d2cfb4a
eccab25415bb7312b6f9c385
ae4f94a892cef7c9b97d8a47
4468b3968d4b7848779867fa
1d5177ad492e614121e3412a


...

ec0bb6a14dc7a35621889c29
4cd0f1469645eb3d3d7f24f5
661eabcab4e1a4f4430a8bf5
dded86b9b792ebb64ab46c5a
338d13c05d82b6304abc31ba
6f810b9b423bf4a9df184208
dcfdf3c1a077f5484273b19a
1e4b6f1a32a8afbba80d1ffb
594534a38e522e13f112e214
67bcc204b72a41923cf57b18
f00cd5386aa04294ee4f925c
80bfcbe36e50e8e0bb4a875b
6bd7353549104727511641c5
33a18045625d7ae3301f3fd0
7ab8de580da3df346560657f
9b0f9bcb485636cf37ce4138
cb7b221ccec65fccfe1c11ff
e5d0c1c55df827266497058f
5ae1f55a3508a50a15e6d96c
4ac54eb95de00d1fa29b07ac
bdf339fe8a575bac1cdaea97
7ff430262aef9a667c1159fe
bcddb9668fcdaacca055ed8f
1923bd6c561227672cb22b37
d5d7f2dd65f5accf9bead060
4ae2a9861ff1d578616544cb
2d57bde994c285390f24d9da
a368492f1603e630d1f6365e
adb0288335a333a023c52450
4160b92ee4c1ff46930e62bb
3a9f0c0706cd544445e285be
ae88d66456d8ce09ec21e961
126cd7fb3b41d60bf1daa09e
200cb11345f3c33e56ab9f7a
e75ab37fa6c2f7033bc1e622
1271439fe6ddd5197cc4302b
831e765935599aae24136694
d18f6b43b237bf01d89e8ff4
46d888eb20320727e7bff138
a725be7898a0974a3786f579


f611b80f0b9ca860ec6b1820
b8412ee1c4f463144366f8b3
222c1d5ad1aaba5c33ed7619
f5dc57a2a8c085b4872979ad
0df61722f821bd582073f8f9
f18ab8ed68f7f6fa65777f27
5bed81bfa36d9f9e93c0287b
d65e506598ccaeea141fd862
8ceda08e8b771cf2a40cf954
19ac67df02f25a13b29085fd
2bd6c21e2cf991d66d573911
3ad79a8d8657a280254670e5
02f48e0c1d3ae320da623465
aad0154a037cf20c9de4c7ce
10edd50e1e841fcb0e051859
b4f7fe98679849bcaef75089
75b774d38e0901a5a6e6f3f9
8163f2885ab9c8cb5c769f57
69d2fa811976737c138b1a66
2ea14592934ace00fa012187
8eca7903caed038376666931
8a74bba57175650f1bf47d2d
b6c0e233809affdedbdc1617
d2da63635c5399e5f85ddc2f
8f7266979011fc6a823c9660
426e99068470569a29e1878c
41e59ca1d9d0e2fb6e323c55
70cfe3b9e067c709578b428e
c6114c3baa3e79910c289d35
2d40d4f8e0890b90f4ce8db7
9a38272c8f930479d265b552
661b567a299d00f490ed1753
d2eaaff3366350067b500300
10158647498389fa30a74596
89f6f0039eb17b7e3fe1e3a1
9b97e7c6a196ecff376f6286
870137885131af2774fbdc10
0a5853561b48fedd242fbc9c
ade976407c4fd3ec2d3a7052
17d64c600635199212834308


b5bff94db60d4d41f05f74d4
93ee87c3a11f14c690385603
62e876511f4cc230947a5aed
52d0a57ee6b83c85154bd76d
faf4b098f989fa77bb9d98ba
c98b2f69cfd2727de58d6dbd
ed8b414e1af56dd1cb0709f3
90dc228d9e58a3f9a9ec53b1
12d9444bcc339d1d5d6fc5de
18dbd3ef5ed9a548b1998f3a
0f0490a888e603867a35058d
b99aeb4383a0542ba4d756a6
5995f77a348de0064175a326
c5356b645f23a4bbc0751168
ae4a00580581d2b4d93a3bf0
6ea26035c8f824f0a53619ec
fc748f5598e7fa70f2a43732
531bb92a09e5921e870ed289
8d6678d38431a0645fd97842
66979b40bcc2cd19e4dff574
eec923f81af818fa94401e7b
781fe2f592e670593be7e270
653dc66f74378efc4ba3e563
71125bf808ad2bd5aee15c03
5c55c8d0c056e2c9057bfc5b
430a2b6a4edca051fe0147b5
efcac82606b2fae6d2a4afd0
eec5addebf516b06f13f5d2e
ad738e7c40484945c6ed1e80
4c037ce18739e5a695310d02
75494292416b67ca253d57eb
bff65b378c7616f8dff5ebd6
555bc0d71d89c19f63150393
80691ca407ace23f457b3990
20c49fbdc9f8c5e529f2c852
244a6ae4171e411fcb0df9f6
cd3c2d5187f482b2027ef11d
fecbd94f4bf0f149059314bb
9e1ef3fbaa41211403c416cb
fbaeab37eff50db1e4db415e


bdc2f42cb3c179d261e9c1dc
4facc62221475cd6231ff110
7d952ede3e538b6dd2542d80
ad7b5989f81e9e814bc6e4e6
9f464713a4dd389e8b34b1cd
3ecec009b82dbba64dbccbfd
f113fd907c58426eaf04bf6c
0daa1ff5e2172ec87ccb4eec
c46ab4ba1b457a19c3ccdc20
50b5667170838432806695db
674e7b76d9c9062f109c88b0
bfc1e316443f4a447f4a0171
3682b9960e854c147d53aa50
b7d0c0028f89b203516c1964
484ff3e315335137085904df
40e9932682810168d1d8633e
753d6c4d7d97a4462935063e
5f8e8df23eaeb6d7a2d09577
3a9443080be9afd68e1ff6ec
48961a73cf3950377d2f82e4
7729eb3ab357e768c7c27cd1
91fed60acde001e48411cfe3
53ebf681d819f0a2b1558e68
c3b711fb69f05c22c8a9b41d
139519c093375bdeef1b4244
e6b5953f9e1c69a8b5deaadc
c1c89da19416ae96687eeb01
0c47b8744c09ba28ce3aeff9
71b5325af393ed5575fe3afe
ee6bd679d0a0c41925b51b4e
76d61d04180f2fd89165e654
b66db6c3e99aa3b13f405a2f
6e3b5e7ab721d172038736ba
594c88862805157d7e827eef
40b036a37d27238982842480
62334fbdd398a21322459c65
a46c4adcbb60a5edfcad81be
b137f53fcad740e9bc53ff07
3b5a975d8701d27e1af82379
69d28b69166e089f5142e145


94a6701cb3f78e4f24244818
0cc39e2c5aa8e27de16c361a
d934a0e75fef27cdef9c7b7c
c405b513f3d729f1afd2c125
eae8c64f843576b5a14eae15
84f50f9436cf6acd31c1c8d8
658294cae7458d1c245c595c
bfe0039b6198ca322054a347
de1f89430ceee8b9c762dd3b
ee061d27fc17b088a307e30d
1f78b64dedf61a0c0a6887b2
7953192b2aeac3b8c241216e
fb2b2f24f4d4d45d01ffb36b
fb5e1037a6aa945895a07afd
9507127f52a63eae839c2783
9591bb175bdfa524d810b454
b14dfbf591d852e8cde7dccc
ef4822577bcad2fca7549d4d
a5af551fc527cf41507e914b
bc3be33d77f00e14f9e44dee
ba3826d8fcec7fb13919be0a
6b41496129754ae2edfb99fe
7c3173ff0d93ea11c0b5042a
3aba6f61f94191125d7e20fe
464aed750c366300eafbab70
375badbbe538402a14d1157a
5b686d6292efb6ed1795bfcf
452cf2ed0f45dd763712b758
ef223f56a3c81028e1f2fcbd
c83f0ef2417b768cf7606f9a
3bc4f3d061f45993d4c7d04a
3ba47591f5dc94f516a67cd3
8f0480759bfa55f620cb9c1c
d29c878da2b257468c186f52
c3b9746532099d04804a5258
6e5b61b698b0f0771f202da4
ee86b9c05ee7058414d40407
4f139ef0c73cd026c639c52d
56482ca4ba4ca07d459297d1
68f842285f9898aebe9fac4c


In [26]:
sample_every_n_visits_options : List[int] = list(range(1, 1000))

true = True
false = False
for sample_every_n_visits in sample_every_n_visits_options:
  if sample_every_n_visits in every_n_visits_to_dev_evaluation_results:
    continue
  parameter_info_list = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_08"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
  for parameter_info in parameter_info_list:
    if parameter_info['name'] == 'sample_every_n_visits':
      parameter_info['values'] = [sample_every_n_visits]
      parameter_info['value'] = sample_every_n_visits
  train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)
  print(sample_every_n_visits)
  evaluation_results = evaluate_baseline_model_on_dataset(parameter_info_list, dev_data)
  every_n_visits_to_dev_evaluation_results[sample_every_n_visits] = evaluation_results
  print(evaluation_results)
  #evaluation_results = evaluate_baseline_model_on_dataset(parameter_info_list, train_data)
  #every_n_visits_to_train_evaluation_results[sample_every_n_visits] = evaluation_results
  #print(evaluation_results)





{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
1
{'dev_correct': 5310, 'dev_total': 6439, 'dev_confusion': [[2426, 218, 45, 57], [294, 2158, 114, 22], [110, 99, 549, 38], [70, 19, 43, 177]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [2], 'value': 2}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_p

12
{'dev_correct': 4793, 'dev_total': 6439, 'dev_confusion': [[2149, 367, 113, 117], [300, 1979, 246, 63], [108, 140, 507, 41], [62, 15, 74, 158]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [13], 'value': 13}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
13
{'dev_correct': 4753, 'dev_total': 6439, 'dev_confusion': [[2145, 350, 129, 122], [330, 1944, 258, 56], [113, 142, 516, 25], [73, 15, 73, 148]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type'

24
{'dev_correct': 4363, 'dev_total': 6439, 'dev_confusion': [[1898, 505, 168, 175], [318, 1860, 321, 89], [113, 172, 473, 38], [66, 13, 98, 132]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [25], 'value': 25}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
25
{'dev_correct': 4249, 'dev_total': 6439, 'dev_confusion': [[1797, 578, 168, 203], [331, 1848, 318, 91], [114, 179, 462, 41], [50, 16, 101, 142]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type

36
{'dev_correct': 4000, 'dev_total': 6439, 'dev_confusion': [[1666, 627, 233, 220], [325, 1758, 406, 99], [115, 195, 460, 26], [58, 16, 119, 116]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [37], 'value': 37}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
37
{'dev_correct': 3995, 'dev_total': 6439, 'dev_confusion': [[1643, 648, 231, 224], [321, 1781, 387, 99], [114, 199, 456, 27], [58, 16, 120, 115]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'typ

48
{'dev_correct': 3856, 'dev_total': 6439, 'dev_confusion': [[1567, 695, 260, 224], [338, 1739, 410, 101], [114, 209, 445, 28], [65, 18, 121, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [49], 'value': 49}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
49
{'dev_correct': 3837, 'dev_total': 6439, 'dev_confusion': [[1551, 707, 262, 226], [339, 1737, 411, 101], [114, 210, 444, 28], [65, 18, 121, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 't

...

948
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [949], 'value': 949}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
949
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

960
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [961], 'value': 961}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
961
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

972
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [973], 'value': 973}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
973
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

984
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [985], 'value': 985}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
985
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

996
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [997], 'value': 997}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
997
{'dev_correct': 3454, 'dev_total': 6439, 'dev_confusion': [[1277, 802, 407, 260], [390, 1651, 446, 101], [114, 233, 421, 28], [48, 18, 138, 105]]}
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits'

In [ ]:
scatterplot = []
for k,v in every_n_visits_to_dev_evaluation_results.items():
  scatterplot.append([k,v['dev_correct'] / v['dev_total']])

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


In [ ]:
print(scatterplot)

In [ ]:


# Create a trace
trace = go.Scatter(
    x = [x[0] for x in scatterplot],
    y = [x[1] for x in scatterplot],
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data)

In [ ]:
baseline_name_and_correctness : List[Tuple[str, float]] = []
for k2,v2 in baseline_name_to_stats.items():
  correctness : float = float(v2['dev_correct'] / v2['dev_total'])
  baseline_name_and_correctness.append((k2, correctness))
print(baseline_name_and_correctness)